In [1]:
import json
import jsonlines


In [2]:
with open("/root/code/ChatGLM_PEFT/main/data/sample_dev_pred.json", 'r', encoding='utf-8') as f:
    obj1 = json.load(f)
    print(type(obj1['answer']))

<class 'dict'>


In [3]:

# /root/code/ChatGLM_PEFT/main/data/hotpot_dev_distractor_v1.json
dev = []
with open("/root/code/ChatGLM_PEFT/main/data/hotpot_dev_distractor_v1.json", 'r', encoding='utf-8') as file:
        dev = json.load(file)
print(dev[0])

{'_id': '5a8b57f25542995d1e6f1371', 'answer': 'yes', 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'supporting_facts': [['Scott Derrickson', 0], ['Ed Wood', 0]], 'context': [['Ed Wood (film)', ['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.', " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.", ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.']], ['Scott Derrickson', ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.', ' He lives in Los Angeles, California.', ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "D

In [22]:
# 打开JSONL文件
# with open("/root/code/ChatGLM_PEFT/output.txt", "r", encoding="utf8") as f:

# 读取 .jsonl 文件并将其转换为 Python 对象列表
from json import JSONDecodeError


def read_jsonl(file_path):    
    data = {
        "answer":{},
        "sp":{}
    }
    with open(file_path, 'r', encoding='utf-8') as file:
        count_index = 0
        while True:
            # 读取两行
            answer_line = json.loads(file.readline().strip())
            sp_line = json.loads(file.readline().strip())
            
            # 如果两行都为空，表示文件已经读完，退出循环
            if not answer_line and not sp_line:
                break
            
            answer = ""
            try:
                answer = json.loads(str(answer_line).strip())  # 逐行读取并转换为 Python 对象
                # print(answer)
                # print("answer_type:"+str(type(answer)))
            except TypeError:
                answer = "<|>解析错误<|>"
                # print("答案解析错误")
            except JSONDecodeError:
                try:
                    clean_line = answer_line.strip().strip('"').strip().strip('"')
                    answer = clean_line
                except Exception:
                    answer = "<|>解析错误<|>"
                    # print("答案解析错误") 

            # print(answer)    
            data['answer'][dev[count_index]['_id']] = str(answer)    
    
            sp=[]
            try:
                sp = json.loads(sp_line)
                # print(sp)
                # print("sp_type"+str(type(sp)))
                # data.append(obj1) 
            except JSONDecodeError :
                # print(sp_line)
                # 绝大多数格式错误都是少了前面的 [
                sp_line = "[" + str(sp_line)
                try:
                    sp = json.loads(sp_line)
                    
                    # print("JSONDecode_sp_type" + str(type(sp)))
                    # data.append(obj1)   
                except JSONDecodeError:
                    #    其他类型错误，直接舍弃
                    sp=[]
            except TypeError:
                sp=[]
            # print(sp)
            data['sp'][dev[count_index]['_id']] = sp
            count_index += 1
    return data

# 使用示例
file_path = '/root/code/ChatGLM_PEFT/output.txt'  # 将其替换为你的 .jsonl 文件路径
data = read_jsonl(file_path)
print(data['sp']['5adc93a4554299438c868e01'])

[['Dupont Circle', 'G0'], ['The Phillips Collection', 'C0']]


In [5]:

# # 打印读取的内容
# for obj in data:
#     print(obj + str(type(obj)))
